In [9]:
myCollection = "Spark The Definitive Guide : Big Data Processing Made Simple"\
    .split(" ")
words = spark.sparkContext.parallelize(myCollection, 2)

ParallelCollectionRDD[6] at readRDDFromFile at PythonRDD.scala:274

In [5]:
words.map(lambda word: (word.lower(), 1))

PythonRDD[3] at RDD at PythonRDD.scala:53

In [6]:
# 단어의 첫번째 문자를 키로 만듬 
keyword = words.keyBy(lambda word: word.lower()[0])

In [7]:
keyword

PythonRDD[4] at RDD at PythonRDD.scala:53

In [8]:
keyword.mapValues(lambda word: word.upper()).collect()

[('s', 'SPARK'),
 ('t', 'THE'),
 ('d', 'DEFINITIVE'),
 ('g', 'GUIDE'),
 (':', ':'),
 ('b', 'BIG'),
 ('d', 'DATA'),
 ('p', 'PROCESSING'),
 ('m', 'MADE'),
 ('s', 'SIMPLE')]

In [10]:
# 반환되는 결과의 각 로우가 문자를 나타내도록 확장
keyword.flatMapValues(lambda word: word.upper()).collect()

[('s', 'S'),
 ('s', 'P'),
 ('s', 'A'),
 ('s', 'R'),
 ('s', 'K'),
 ('t', 'T'),
 ('t', 'H'),
 ('t', 'E'),
 ('d', 'D'),
 ('d', 'E'),
 ('d', 'F'),
 ('d', 'I'),
 ('d', 'N'),
 ('d', 'I'),
 ('d', 'T'),
 ('d', 'I'),
 ('d', 'V'),
 ('d', 'E'),
 ('g', 'G'),
 ('g', 'U'),
 ('g', 'I'),
 ('g', 'D'),
 ('g', 'E'),
 (':', ':'),
 ('b', 'B'),
 ('b', 'I'),
 ('b', 'G'),
 ('d', 'D'),
 ('d', 'A'),
 ('d', 'T'),
 ('d', 'A'),
 ('p', 'P'),
 ('p', 'R'),
 ('p', 'O'),
 ('p', 'C'),
 ('p', 'E'),
 ('p', 'S'),
 ('p', 'S'),
 ('p', 'I'),
 ('p', 'N'),
 ('p', 'G'),
 ('m', 'M'),
 ('m', 'A'),
 ('m', 'D'),
 ('m', 'E'),
 ('s', 'S'),
 ('s', 'I'),
 ('s', 'M'),
 ('s', 'P'),
 ('s', 'L'),
 ('s', 'E')]

In [13]:
# 키 추출
keyword.keys().collect()

['s', 't', 'd', 'g', ':', 'b', 'd', 'p', 'm', 's']

In [14]:
# 값 추출
keyword.values().collect()

['Spark',
 'The',
 'Definitive',
 'Guide',
 ':',
 'Big',
 'Data',
 'Processing',
 'Made',
 'Simple']

In [15]:
# 특정 키에 관한 결과 찾기
keyword.lookup("s")

['Spark', 'Simple']

In [16]:
# 근사치, 정확도를 이용해 키를 기반으로 RDD 샘플을 생성
import random

distinctChars = words.flatMap(lambda word: list(word.lower())).distinct().collect()
sampleMap = dict(map(lambda c: (c, random.random()), distinctChars))
words.map(lambda word: (word.lower()[0], word)).sampleByKey(True, sampleMap, 6).collect()

[('t', 'The'), ('t', 'The'), ('d', 'Definitive'), (':', ':')]

In [17]:
chars = words.flatMap(lambda word: word.lower())
KVcharacters = chars.map(lambda letter: (letter, 1))

def maxFunc(left, right):
    return max(left, right)
def addFunc(left, right):
    return left+right

nums = sc.parallelize(range(1, 31), 5)

In [19]:
# 각 키의 아이템 수를 구하고 로컬 맵으로 결과 수집
KVcharacters.countByKey()

defaultdict(int,
            {'s': 4,
             'p': 3,
             'a': 4,
             'r': 2,
             'k': 1,
             't': 3,
             'h': 1,
             'e': 7,
             'd': 4,
             'f': 1,
             'i': 7,
             'n': 2,
             'v': 1,
             'g': 3,
             'u': 1,
             ':': 1,
             'b': 1,
             'o': 1,
             'c': 1,
             'm': 2,
             'l': 1})

In [23]:
# groupByKey

import functools.reduce

KVcharacters.groupByKey().map(lambda row: (row[0], reduce(addFunc, row[1]))).collect()


ModuleNotFoundError: No module named 'functools.reduce'; 'functools' is not a package

In [24]:
KVcharacters.reduceByKey(addFunc).collect()

[('s', 4),
 ('p', 3),
 ('r', 2),
 ('h', 1),
 ('d', 4),
 ('i', 7),
 ('g', 3),
 ('b', 1),
 ('c', 1),
 ('l', 1),
 ('a', 4),
 ('k', 1),
 ('t', 3),
 ('e', 7),
 ('f', 1),
 ('n', 2),
 ('v', 1),
 ('u', 1),
 (':', 1),
 ('o', 1),
 ('m', 2)]

In [25]:
# aggregate(파티션 내에서 수행되는 함수, 모든 파티션에 걸쳐 수행)
# aggregate -> null 값이나 집계의 시작값이 필요

nums.aggregate(0, maxFunc, addFunc)

90

In [26]:
depth = 3
nums.treeAggregate(0, maxFunc, addFunc, depth)

90

In [27]:
# 파티션 대신 키를 기준으로 연산 수행
KVcharacters.aggregateByKey(0, addFunc, maxFunc).collect()

[('s', 3),
 ('p', 2),
 ('r', 1),
 ('h', 1),
 ('d', 2),
 ('i', 4),
 ('g', 2),
 ('b', 1),
 ('c', 1),
 ('l', 1),
 ('a', 3),
 ('k', 1),
 ('t', 2),
 ('e', 4),
 ('f', 1),
 ('n', 1),
 ('v', 1),
 ('u', 1),
 (':', 1),
 ('o', 1),
 ('m', 2)]

In [28]:
# 집계함수 대신 컴바이너 사용
def valToCombiner(value):
    return [value]
def mergeValuesFunc(vals, valToAppend):
    vals.append(valToAppend)
    return vals
def mergeCombinerFunc(vals1, vals2):
    return vals1 + vals2
outputPartitions = 6

KVcharacters\
.combineByKey(
    valToCombiner,
    mergeValuesFunc,
    mergeCombinerFunc,
outputPartitions).collect()

[('s', [1, 1, 1, 1]),
 ('d', [1, 1, 1, 1]),
 ('l', [1]),
 ('v', [1]),
 (':', [1]),
 ('p', [1, 1, 1]),
 ('r', [1, 1]),
 ('c', [1]),
 ('k', [1]),
 ('t', [1, 1, 1]),
 ('n', [1, 1]),
 ('u', [1]),
 ('o', [1]),
 ('h', [1]),
 ('i', [1, 1, 1, 1, 1, 1, 1]),
 ('g', [1, 1, 1]),
 ('b', [1]),
 ('a', [1, 1, 1, 1]),
 ('e', [1, 1, 1, 1, 1, 1, 1]),
 ('f', [1]),
 ('m', [1, 1])]

In [29]:
# 각 키의 값 병합(덧셈 -> 0)
KVcharacters.foldByKey(0, addFunc).collect()

[('s', 4),
 ('p', 3),
 ('r', 2),
 ('h', 1),
 ('d', 4),
 ('i', 7),
 ('g', 3),
 ('b', 1),
 ('c', 1),
 ('l', 1),
 ('a', 4),
 ('k', 1),
 ('t', 3),
 ('e', 7),
 ('f', 1),
 ('n', 2),
 ('v', 1),
 ('u', 1),
 (':', 1),
 ('o', 1),
 ('m', 2)]

In [30]:
# RDD를 그룹화, 각 키를 기준으로 값을 결합(그룹 기반의 조인)

import random

distinctChars = words.flatMap(lambda word: word.lower()).distinct()
charRDD = distinctChars.map(lambda c: (c, random.random()))
charRDD2 = distinctChars.map(lambda c: (c, random.random()))

charRDD.cogroup(charRDD2).take(5)

[('s',
  (<pyspark.resultiterable.ResultIterable at 0x7f9bdddee040>,
   <pyspark.resultiterable.ResultIterable at 0x7f9bdddee100>)),
 ('p',
  (<pyspark.resultiterable.ResultIterable at 0x7f9bdddee310>,
   <pyspark.resultiterable.ResultIterable at 0x7f9bdddee550>)),
 ('r',
  (<pyspark.resultiterable.ResultIterable at 0x7f9bdddee5b0>,
   <pyspark.resultiterable.ResultIterable at 0x7f9bdddb29d0>)),
 ('i',
  (<pyspark.resultiterable.ResultIterable at 0x7f9bdddb2670>,
   <pyspark.resultiterable.ResultIterable at 0x7f9bdddb2460>)),
 ('g',
  (<pyspark.resultiterable.ResultIterable at 0x7f9bdddb22b0>,
   <pyspark.resultiterable.ResultIterable at 0x7f9bdddb2ee0>))]

In [31]:
# 내부 조인

keyedChars = distinctChars.map(lambda c: (c, random.random()))
outputPartitions = 10

KVcharacters.join(keyedChars).count()
KVcharacters.join(keyedChars, outputPartitions).count()

51

In [32]:
# zip

numRange = sc.parallelize(range(10), 2)
words.zip(numRange).collect()

[('Spark', 0),
 ('The', 1),
 ('Definitive', 2),
 ('Guide', 3),
 (':', 4),
 ('Big', 5),
 ('Data', 6),
 ('Processing', 7),
 ('Made', 8),
 ('Simple', 9)]

In [33]:
# coalesce: 파티션을 재분배할 때 발생하는 데이터 셔플을 방지하기 위해 동일한 워커에 존재하는 파티션을 합치는 메서드

# 데이터 셔플링 없이 하나의 파티션으로 합치기
words.coalesce(1).getNumPartitions()

1

In [34]:
words.repartition(10)

MapPartitionsRDD[91] at coalesce at NativeMethodAccessorImpl.java:0

In [35]:
df = spark.read.option("header", "true").option("inferSchema", "true")\
.csv("./data/retail-data/all/")
rdd = df.coalesce(10).rdd

df.printSchema()

root
 |-- InvoiceNo: string (nullable = true)
 |-- StockCode: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- InvoiceDate: string (nullable = true)
 |-- UnitPrice: double (nullable = true)
 |-- CustomerID: integer (nullable = true)
 |-- Country: string (nullable = true)



In [36]:
def partitionFunc(key):
    import random
    
    if key == 17850 or key == 12583:
        return 0
    else:
        return random.randint(1,2)
    
keyedRDD = rdd.keyBy(lambda row: row[6])

keyedRDD\
.partitionBy(3, partitionFunc)\
.map(lambda x: x[0])\
.glom()\
.map(lambda x: len(set(x)))\
.take(5)

[2, 4308, 4295]